In [2]:
def get_mapping_relation(height, width, a=0.0, b=0, c=-0.2, d=1.0):
    """
    Args:
        image(np.array, [H, W, 3]): original image
        a(float): affects only the outermost pixels of the image(x^4)
        b(float): most cases only require b optimization(x^3)
        c(float): most uniform correction(x^2)
        d(float): describes the linear scaling of the image(x)
    Returns:
        location_of_dest_image(np.array): destination of the original image index
    """
    x, y = np.meshgrid(range(width), range(height))
    x = x.reshape(-1)
    y = y.reshape(-1)
    location_of_source_image = np.stack([x, y], 1)

    center_x = x.mean()
    center_y = y.mean()
    center = np.array([center_x, center_y])
    norm = np.linalg.norm(center)
    dist = np.sqrt(((location_of_source_image - center) ** 2).sum(1))
    r = np.sqrt(((x - center_x) / norm) ** 2 + ((y - center_y) / norm) ** 2) # r=dist/norm, r in [0, 1]
    rdest = (a * r ** 4 + b * r ** 3 + c * r ** 2 + d * r) * norm
    target_x = rdest / dist * (x - center_x) + center_x
    target_y = rdest / dist * (y - center_y) + center_y
    location_of_dest_image = np.stack([target_x, target_y], 1)
    return location_of_dest_image.reshape((-1, 2))

def map_image(image, map_array):
    height, width = image.shape[:2]
    map_x = map_array[:, 1].reshape((height, width)).astype(np.float32)
    map_y = map_array[:, 0].reshape((height, width)).astype(np.float32)
    mapped_img = cv2.remap(image, map_y, map_x, cv2.INTER_LINEAR)
    return mapped_img

def map_xy(x, y, x_c=639.5, y_c=359.5, c=-0.2, d=1):
    norm = np.linalg.norm((x_c, y_c))
    x_norm = (x - x_c)/norm
    y_norm = (y- y_c)/norm
    dist_relative = np.sqrt(x_norm**2 + y_norm**2)
    dist_map_relative = c*dist_relative**2 + d*dist_relative
    ratio = dist_map_relative/dist_relative # this is c*dist_relative + d
    xm = ratio*(x-x_c) + x_c
    ym = ratio*(y-y_c) + y_c
    return xm, ym
    


# 解读
get_mapping_relation的点[0,0]的数值表示在畸变后的图里这个点出现的位置, 所以在畸变后的图里去找这个点的数值就是这个点的数值，但是往往这个点并不是整数，所以需要进行插值：
$$
dst(x,y) = src(map\_xy(x,y))
$$
where src is distort image, dst is the desire image.
map_xy是get_mapping_relation的单点版本。
我们可以写出$(map\_xy)^{-1}$:  
```
x = g(xm, ym)...  
y = h(ym, ym)... 
set:
xm = e
ym = f
xc = g
yc = h
y/x = ym/xm # since they are on the same line
solve (f-h)/(y-h) = d + c sqrt(((y e - g f)/nf)^2 + ((y - g)/n)^2) in wolfram alpha gives CRAZY long result , mayby I am wrong
```